Here we'll try the UNGDC

In [2]:
from gensim.models.fasttext import FastText
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import string

In [4]:
un_model = FastText(size=200)

2019-11-08 00:20:45,987 : INFO : resetting layer weights


In [8]:
import re
import nltk
stopwords = set(nltk.corpus.stopwords.words('english')) 
def preprocess(sent):
    # Remember: sent is a list of tokens
    to_return = [t.lower() for t in sent]
    to_return = [t.translate(str.maketrans('', '', string.punctuation)) for t in to_return]
    to_return = [t for t in to_return if t not in stopwords]
    to_return = [re.sub(r'\d+', '', t) for t in to_return if t != ""]
    return to_return

In [6]:
import pandas as pd
import numpy as np
un_fpath = "texts/UNGDC.csv"
un_df = pd.read_csv(un_fpath)

In [7]:
un_df.head()

,id,folder_name,file_name,file_prefix,session_num,year,country_code,text
0,0,Session 25 - 1970,ALB_25_1970.txt,ALB_25_1970,25,1970,ALB,33: May I first convey to our President the co...
1,1,Session 25 - 1970,ARG_25_1970.txt,ARG_25_1970,25,1970,ARG,177.\t : It is a fortunate coincidence that pr...
2,2,Session 25 - 1970,AUS_25_1970.txt,AUS_25_1970,25,1970,AUS,100.\t It is a pleasure for me to extend to y...
3,3,Session 25 - 1970,AUT_25_1970.txt,AUT_25_1970,25,1970,AUT,155.\t May I begin by expressing to Ambassado...
4,4,Session 25 - 1970,BEL_25_1970.txt,BEL_25_1970,25,1970,BEL,"176. No doubt each of us, before coming up to ..."


In [17]:
re_tokenizer = re.compile(r'\w+')

def parse_text(text):
    # First tokenize the full text down into a list of sentences, then call
    # preprocess() on each sentence
    text_tokens = re_tokenizer.findall(text)
    return text_tokens

In [19]:
# Sampling 500 speeches at random
sample_df = un_df.sample(20)
all_sents = []
num_texts = len(sample_df)
for text_num, cur_text in enumerate(sample_df["text"]):
    # Making sure we're progressing
    #if text_num % 100 == 0:
    print(f"Processing text #{text_num} of {num_texts}")
    text_sents = [parse_text(t) for t in un_df["text"]]
    # extend() rather than append() since we *don't* want a list-of-lists
    all_sents.extend(text_sents)

Processing text #0 of 20
Processing text #1 of 20
Processing text #2 of 20
Processing text #3 of 20
Processing text #4 of 20
Processing text #5 of 20
Processing text #6 of 20
Processing text #7 of 20
Processing text #8 of 20
Processing text #9 of 20
Processing text #10 of 20
Processing text #11 of 20
Processing text #12 of 20
Processing text #13 of 20
Processing text #14 of 20
Processing text #15 of 20
Processing text #16 of 20
Processing text #17 of 20
Processing text #18 of 20
Processing text #19 of 20


In [20]:
un_model.build_vocab(all_sents)

2019-11-08 01:51:35,061 : INFO : collecting all words and their counts
2019-11-08 01:51:35,069 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-08 01:51:53,472 : INFO : PROGRESS: at sentence #10000, processed 31046028 words, keeping 65318 word types
2019-11-08 01:52:09,832 : INFO : PROGRESS: at sentence #20000, processed 60098082 words, keeping 65318 word types
2019-11-08 01:52:24,108 : INFO : PROGRESS: at sentence #30000, processed 86919903 words, keeping 65318 word types
2019-11-08 01:52:39,708 : INFO : PROGRESS: at sentence #40000, processed 114778067 words, keeping 65318 word types
2019-11-08 01:52:56,863 : INFO : PROGRESS: at sentence #50000, processed 145546966 words, keeping 65318 word types
2019-11-08 01:53:12,965 : INFO : PROGRESS: at sentence #60000, processed 174065901 words, keeping 65318 word types
2019-11-08 01:53:34,751 : INFO : PROGRESS: at sentence #70000, processed 200646333 words, keeping 65318 word types
2019-11-08 01:53:56,710 : IN

In [21]:
print(sum(1 for _ in all_sents))
print(sum(1 for _ in all_sents))
print(sum(1 for _ in all_sents))

157940
157940
157940


In [24]:
# train the model
vocab_size = len(un_model.wv.vocab)
un_model.train(
    sentences=all_sents, epochs=un_model.epochs,
    total_examples=un_model.corpus_count, total_words=vocab_size
)

2019-11-08 02:22:06,332 : INFO : training model with 3 workers on 65318 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-11-08 02:22:07,401 : INFO : EPOCH 1 - PROGRESS: at 0.02% examples, 73323 words/s, in_qsize 6, out_qsize 1
2019-11-08 02:22:08,440 : INFO : EPOCH 1 - PROGRESS: at 0.04% examples, 80857 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:22:09,483 : INFO : EPOCH 1 - PROGRESS: at 0.06% examples, 84590 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:22:10,528 : INFO : EPOCH 1 - PROGRESS: at 0.08% examples, 86462 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:22:11,542 : INFO : EPOCH 1 - PROGRESS: at 0.10% examples, 87493 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:22:12,563 : INFO : EPOCH 1 - PROGRESS: at 0.12% examples, 87296 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:22:13,604 : INFO : EPOCH 1 - PROGRESS: at 0.14% examples, 87819 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:22:14,689 : INFO : EPOCH 1 - PROGRESS: at 0.16% examples, 8

2019-11-08 02:23:24,040 : INFO : EPOCH 1 - PROGRESS: at 1.44% examples, 85758 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:23:25,049 : INFO : EPOCH 1 - PROGRESS: at 1.47% examples, 85818 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:23:26,103 : INFO : EPOCH 1 - PROGRESS: at 1.49% examples, 85808 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:23:27,108 : INFO : EPOCH 1 - PROGRESS: at 1.51% examples, 85945 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:23:28,139 : INFO : EPOCH 1 - PROGRESS: at 1.53% examples, 85965 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:23:29,169 : INFO : EPOCH 1 - PROGRESS: at 1.56% examples, 85988 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:23:30,205 : INFO : EPOCH 1 - PROGRESS: at 1.58% examples, 86000 words/s, in_qsize 4, out_qsize 1
2019-11-08 02:23:31,258 : INFO : EPOCH 1 - PROGRESS: at 1.60% examples, 86014 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:23:32,297 : INFO : EPOCH 1 - PROGRESS: at 1.63% examples, 86077 words/s, in_qsize 6, out_qsize 0
2

2019-11-08 02:24:42,085 : INFO : EPOCH 1 - PROGRESS: at 3.70% examples, 86324 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:24:43,120 : INFO : EPOCH 1 - PROGRESS: at 3.74% examples, 86348 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:24:44,187 : INFO : EPOCH 1 - PROGRESS: at 3.78% examples, 86362 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:24:45,231 : INFO : EPOCH 1 - PROGRESS: at 3.82% examples, 86333 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:24:46,234 : INFO : EPOCH 1 - PROGRESS: at 3.87% examples, 86360 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:24:47,243 : INFO : EPOCH 1 - PROGRESS: at 3.91% examples, 86382 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:24:48,281 : INFO : EPOCH 1 - PROGRESS: at 3.95% examples, 86408 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:24:49,336 : INFO : EPOCH 1 - PROGRESS: at 3.98% examples, 86377 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:24:50,411 : INFO : EPOCH 1 - PROGRESS: at 4.02% examples, 86413 words/s, in_qsize 5, out_qsize 0
2

2019-11-08 02:25:59,659 : INFO : EPOCH 1 - PROGRESS: at 5.81% examples, 86943 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:26:00,686 : INFO : EPOCH 1 - PROGRESS: at 5.83% examples, 86967 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:26:01,714 : INFO : EPOCH 1 - PROGRESS: at 5.85% examples, 86970 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:26:02,788 : INFO : EPOCH 1 - PROGRESS: at 5.87% examples, 86982 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:26:03,809 : INFO : EPOCH 1 - PROGRESS: at 5.89% examples, 87008 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:26:04,852 : INFO : EPOCH 1 - PROGRESS: at 5.91% examples, 87025 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:26:05,893 : INFO : EPOCH 1 - PROGRESS: at 5.93% examples, 86962 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:26:06,899 : INFO : EPOCH 1 - PROGRESS: at 5.94% examples, 86859 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:26:07,986 : INFO : EPOCH 1 - PROGRESS: at 5.95% examples, 86765 words/s, in_qsize 6, out_qsize 0
2

2019-11-08 02:27:17,292 : INFO : EPOCH 1 - PROGRESS: at 7.49% examples, 86606 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:27:18,311 : INFO : EPOCH 1 - PROGRESS: at 7.52% examples, 86618 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:27:19,423 : INFO : EPOCH 1 - PROGRESS: at 7.55% examples, 86607 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:27:20,468 : INFO : EPOCH 1 - PROGRESS: at 7.58% examples, 86626 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:27:21,534 : INFO : EPOCH 1 - PROGRESS: at 7.61% examples, 86633 words/s, in_qsize 5, out_qsize 0
2019-11-08 02:27:22,616 : INFO : EPOCH 1 - PROGRESS: at 7.64% examples, 86644 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:27:23,747 : INFO : EPOCH 1 - PROGRESS: at 7.68% examples, 86635 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:27:24,773 : INFO : EPOCH 1 - PROGRESS: at 7.71% examples, 86660 words/s, in_qsize 6, out_qsize 0
2019-11-08 02:27:25,871 : INFO : EPOCH 1 - PROGRESS: at 7.74% examples, 86654 words/s, in_qsize 6, out_qsize 0
2

...

2019-11-08 07:34:26,391 : INFO : EPOCH 5 - PROGRESS: at 90.43% examples, 86346 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:34:27,418 : INFO : EPOCH 5 - PROGRESS: at 90.45% examples, 86345 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:34:28,509 : INFO : EPOCH 5 - PROGRESS: at 90.47% examples, 86345 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:34:29,539 : INFO : EPOCH 5 - PROGRESS: at 90.49% examples, 86347 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:34:30,570 : INFO : EPOCH 5 - PROGRESS: at 90.51% examples, 86350 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:34:31,588 : INFO : EPOCH 5 - PROGRESS: at 90.53% examples, 86350 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:34:32,619 : INFO : EPOCH 5 - PROGRESS: at 90.55% examples, 86352 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:34:33,690 : INFO : EPOCH 5 - PROGRESS: at 90.57% examples, 86354 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:34:34,706 : INFO : EPOCH 5 - PROGRESS: at 90.59% examples, 86354 words/s, in_qsize 6, out_

2019-11-08 07:35:44,066 : INFO : EPOCH 5 - PROGRESS: at 92.02% examples, 86383 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:35:45,111 : INFO : EPOCH 5 - PROGRESS: at 92.04% examples, 86381 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:35:46,257 : INFO : EPOCH 5 - PROGRESS: at 92.07% examples, 86383 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:35:47,270 : INFO : EPOCH 5 - PROGRESS: at 92.10% examples, 86385 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:35:48,286 : INFO : EPOCH 5 - PROGRESS: at 92.13% examples, 86386 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:35:49,294 : INFO : EPOCH 5 - PROGRESS: at 92.15% examples, 86385 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:35:50,421 : INFO : EPOCH 5 - PROGRESS: at 92.18% examples, 86385 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:35:51,545 : INFO : EPOCH 5 - PROGRESS: at 92.21% examples, 86386 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:35:52,597 : INFO : EPOCH 5 - PROGRESS: at 92.23% examples, 86387 words/s, in_qsize 6, out_

2019-11-08 07:37:03,457 : INFO : EPOCH 5 - PROGRESS: at 94.63% examples, 86361 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:37:04,534 : INFO : EPOCH 5 - PROGRESS: at 94.67% examples, 86363 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:37:05,560 : INFO : EPOCH 5 - PROGRESS: at 94.70% examples, 86361 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:37:06,744 : INFO : EPOCH 5 - PROGRESS: at 94.74% examples, 86355 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:37:07,848 : INFO : EPOCH 5 - PROGRESS: at 94.78% examples, 86356 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:37:08,851 : INFO : EPOCH 5 - PROGRESS: at 94.82% examples, 86359 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:37:10,055 : INFO : EPOCH 5 - PROGRESS: at 94.86% examples, 86356 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:37:11,132 : INFO : EPOCH 5 - PROGRESS: at 94.90% examples, 86355 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:37:12,298 : INFO : EPOCH 5 - PROGRESS: at 94.94% examples, 86355 words/s, in_qsize 5, out_

2019-11-08 07:38:21,375 : INFO : EPOCH 5 - PROGRESS: at 96.31% examples, 86432 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:38:22,414 : INFO : EPOCH 5 - PROGRESS: at 96.33% examples, 86434 words/s, in_qsize 4, out_qsize 1
2019-11-08 07:38:23,450 : INFO : EPOCH 5 - PROGRESS: at 96.35% examples, 86434 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:38:24,523 : INFO : EPOCH 5 - PROGRESS: at 96.37% examples, 86433 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:38:25,563 : INFO : EPOCH 5 - PROGRESS: at 96.39% examples, 86434 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:38:26,570 : INFO : EPOCH 5 - PROGRESS: at 96.42% examples, 86436 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:38:27,646 : INFO : EPOCH 5 - PROGRESS: at 96.44% examples, 86436 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:38:28,666 : INFO : EPOCH 5 - PROGRESS: at 96.46% examples, 86437 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:38:29,710 : INFO : EPOCH 5 - PROGRESS: at 96.49% examples, 86438 words/s, in_qsize 6, out_

2019-11-08 07:39:39,300 : INFO : EPOCH 5 - PROGRESS: at 98.42% examples, 86434 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:39:40,442 : INFO : EPOCH 5 - PROGRESS: at 98.47% examples, 86435 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:39:41,607 : INFO : EPOCH 5 - PROGRESS: at 98.52% examples, 86436 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:39:42,681 : INFO : EPOCH 5 - PROGRESS: at 98.56% examples, 86437 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:39:43,823 : INFO : EPOCH 5 - PROGRESS: at 98.61% examples, 86438 words/s, in_qsize 6, out_qsize 1
2019-11-08 07:39:44,845 : INFO : EPOCH 5 - PROGRESS: at 98.65% examples, 86439 words/s, in_qsize 5, out_qsize 0
2019-11-08 07:39:45,851 : INFO : EPOCH 5 - PROGRESS: at 98.69% examples, 86439 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:39:46,930 : INFO : EPOCH 5 - PROGRESS: at 98.73% examples, 86438 words/s, in_qsize 6, out_qsize 0
2019-11-08 07:39:47,983 : INFO : EPOCH 5 - PROGRESS: at 98.77% examples, 86435 words/s, in_qsize 5, out_

In [25]:
un_model.save("un_fasttext.model")

2019-11-08 13:50:16,282 : INFO : saving FastText object under un_fasttext.model, separately None
2019-11-08 13:50:16,300 : INFO : storing np array 'vectors' to un_fasttext.model.wv.vectors.npy
2019-11-08 13:50:16,593 : INFO : storing np array 'vectors_vocab' to un_fasttext.model.wv.vectors_vocab.npy
2019-11-08 13:50:16,936 : INFO : storing np array 'vectors_ngrams' to un_fasttext.model.wv.vectors_ngrams.npy
2019-11-08 13:50:31,639 : INFO : not storing attribute vectors_norm
2019-11-08 13:50:31,679 : INFO : not storing attribute vectors_vocab_norm
2019-11-08 13:50:31,680 : INFO : not storing attribute vectors_ngrams_norm
2019-11-08 13:50:31,681 : INFO : not storing attribute buckets_word
2019-11-08 13:50:31,685 : INFO : storing np array 'syn1neg' to un_fasttext.model.trainables.syn1neg.npy
2019-11-08 13:50:31,959 : INFO : storing np array 'vectors_vocab_lockf' to un_fasttext.model.trainables.vectors_vocab_lockf.npy
2019-11-08 13:50:32,266 : INFO : storing np array 'vectors_ngrams_lockf'

In [26]:
un_model.wv.save("un_fasttext_vectors.vec")

2019-11-08 13:50:54,153 : INFO : saving FastTextKeyedVectors object under un_fasttext_vectors.vec, separately None
2019-11-08 13:50:54,155 : INFO : storing np array 'vectors' to un_fasttext_vectors.vec.vectors.npy
2019-11-08 13:50:55,342 : INFO : storing np array 'vectors_vocab' to un_fasttext_vectors.vec.vectors_vocab.npy
2019-11-08 13:50:56,199 : INFO : storing np array 'vectors_ngrams' to un_fasttext_vectors.vec.vectors_ngrams.npy
2019-11-08 13:51:10,578 : INFO : not storing attribute vectors_norm
2019-11-08 13:51:10,579 : INFO : not storing attribute vectors_vocab_norm
2019-11-08 13:51:10,579 : INFO : not storing attribute vectors_ngrams_norm
2019-11-08 13:51:10,580 : INFO : not storing attribute buckets_word
2019-11-08 13:51:10,715 : INFO : saved un_fasttext_vectors.vec


t-SNE time

In [29]:
labels = []
tokens = []

for word in un_model.wv.vocab:
    tokens.append(un_model.wv[word])
    labels.append(word)

In [31]:
from sklearn.manifold import TSNE
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
new_values = tsne_model.fit_transform(tokens)

In [ ]:
un_model.wv.most_similar('kissinger', topn=20)

In [ ]:
my_x = new_values[:,0]
my_y = new_values[:,1]
plt.figure(figsize=(32, 32))
#ax = sns.scatterplot(x=my_x,y=my_y)

#plt.annotate(labels,xy=(my_x, my_y),xytext=(5, 2),textcoords='offset points',ha='right',va='bottom')
# control x and y limits
#plt.ylim(-10, 10)
#plt.xlim(-10, 10)

# Randomly sample... 100 points and label them?
a = pd.DataFrame({'x': my_x, 'y': my_y, 'val': labels})
a_sample = a.sample(1000)
counter = 0
for i, point in a_sample.iterrows():
    plt.scatter(point['x'], point['y'])
    plt.text(point['x']+.02, point['y'], str(point['val']))
    counter = counter + 1
        
plt.show()

un_model.most_similar('Trump')

In [33]:
un_model.most_similar('Trump')

C:\Users\jj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2019-11-08 16:05:31,080 : INFO : precomputing L2-norms of word weight vectors
2019-11-08 16:05:31,143 : INFO : precomputing L2-norms of ngram weight vectors


[('rump', 0.613652765750885),
 ('Truman', 0.5916858315467834),
 ('Jump', 0.5739368796348572),
 ('Clinton', 0.5646244287490845),
 ('Dump', 0.5635473728179932),
 ('Bush', 0.5612047910690308),
 ('Obama', 0.5493463277816772),
 ('Cartel', 0.5461411476135254),
 ('Reagan', 0.5381147861480713),
 ('Reaganís', 0.5343747138977051)]